In [22]:
# import os
# api_key = os.getenv("OPENAI_API_KEY")

## Import input data

In [1]:
def read_file(file_path):
    with open(file_path) as f:
        return f.readlines()

# Read job_info.txt,profile_info.txt,letter_raw.txt files
job_info = read_file("jobs/example.txt")
profile_info = read_file("profile_info.txt")
letter_raw = read_file("letter_raw.txt")

## Open AI API

### Function to call the OpenAI API

In [2]:
def create_openai_completion(model, messages):
    from openai import OpenAI
    client = OpenAI()
    try:
        completion = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature= 0.15,
            response_format={ "type": "json_object" }
        )
        return completion
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

### Function to prepare the prompt for the API

In [3]:

def prepare_messages(profile_info, job_info, letter_format):
    return [
        {"role": "system", "content": "Je suis un assistant qui permet de remplir les lettres de motivation pour vos candidatures. Je fais le lien entre vos expériences, vos compétences et l'offre d'emploi. Le schema JSON doit contenir : Entreprise, Poste, Objet, Contenu"},
        {"role": "user", "content": f"Voici mes compétences et expériences : {profile_info}"},
        {"role": "system", "content": "Donnez moi l'offre d'emploi."},
        {"role": "user", "content": f"Voici le contenu de l'offre d'emploi : {job_info}"},
        {"role": "system", "content": "Donne moi le format type de lettre de motivation que je dois suivre."},
        {"role": "user", "content": f"Voici le format type de contenu que j'aimerais que tu suives : {letter_format}"},
    ]

### Function to get the response from the API

In [4]:
import json
messages = prepare_messages(profile_info, job_info, letter_raw)
# completion = create_openai_completion("gpt-4-0125-preview", messages)
completion = create_openai_completion("gpt-3.5-turbo-0125", messages)

if completion:
    result_raw = json.loads(completion.choices[0].message.content)
    print(result_raw)
else:
    print("Failed to get a response from the OpenAI API.")

{'Entreprise': 'Bouygues Telecom', 'Poste': 'Développeur IA/Test', 'Objet': 'Objet : Candidature pour une alternance en Développeur IA/Test chez Bouygues Telecom', 'Contenu': "Madame, Monsieur,\n\nActuellement en préparation d'un Master 2 spécialisé en Big Data & Machine Learning à l'Efrei, je suis activement à la recherche d'une entreprise pour m'accueillir un an à partir d’octobre 2024. Cet expérience sera l’occasion d’enrichir ma formation théorique par une expérience pratique. C'est avec beaucoup d'enthousiasme que je vous présente ma candidature pour une alternance chez Bouygues Telecom pour le poste de Développeur IA/Test.\n\nVotre entreprise est reconnue pour son expertise en matière de technologies de pointe et je suis convaincu que votre entreprise est l'endroit idéal pour développer mes compétences. Bouygues Telecom recrute son/sa futur.e Développeur IA/Test en alternance, et je suis convaincu que je pourrais apporter une contribution significative à votre équipe.\n\nEn tant 

In [5]:
link_to_the_job = job_info[0]
export_content = f"{link_to_the_job}\n\n{result_raw['Objet']}\n{result_raw['Contenu']}"
export_content

"https://bouyguestelecom-recrute.talent-soft.com/offre-de-emploi/emploi-alternance-developpeur-ia-test-f-h_47683.aspx\n\n\nObjet : Candidature pour une alternance en Développeur IA/Test chez Bouygues Telecom\nMadame, Monsieur,\n\nActuellement en préparation d'un Master 2 spécialisé en Big Data & Machine Learning à l'Efrei, je suis activement à la recherche d'une entreprise pour m'accueillir un an à partir d’octobre 2024. Cet expérience sera l’occasion d’enrichir ma formation théorique par une expérience pratique. C'est avec beaucoup d'enthousiasme que je vous présente ma candidature pour une alternance chez Bouygues Telecom pour le poste de Développeur IA/Test.\n\nVotre entreprise est reconnue pour son expertise en matière de technologies de pointe et je suis convaincu que votre entreprise est l'endroit idéal pour développer mes compétences. Bouygues Telecom recrute son/sa futur.e Développeur IA/Test en alternance, et je suis convaincu que je pourrais apporter une contribution signific

## Export result

In [60]:
# Write the content to CL/CL - [Position] - [Company].txt
file_path = f"CL/{result_raw['Entreprise']} - {result_raw['Poste'].replace('/', '-')}.txt"
with open(file_path, "w") as file:
    file.write(export_content)